In [ ]:
import pandas as pd 
import re
from nltk.tokenize import sent_tokenize
from hazm import SentenceTokenizer
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

In [62]:
with open('kb__info.txt', "r", encoding="utf-8") as f:
    text = f.read()

In [63]:
text

' معرفی سرویس ابری رایابات: دستیار هوشمند فروش گفتگومحور مبتنی بر مدل\u200cهای زبانی بزرگ   \n\nشرکت شما به دنبال تحول در فرآیندهای فروش و تعامل با مشتریان است؟  رایابات  به عنوان یک سرویس ابری پیشرفته، امکان طراحی و پیاده\u200cسازی  عامل\u200cهای هوشمند فروش گفتگومحور  را با استفاده از آخرین فناوری\u200cهای پردازش زبان طبیعی (NLP) و مدل\u200cهای زبانی بزرگ (LLM) فراهم می\u200cکند. این راهکار، نه تنها تعامل با مشتریان را بهینه می\u200cسازد، بلکه با ابزارهای تحلیلی قدرتمند و داشبوردهای کاربردی، مدیریت فرآیند فروش را متحول می\u200cکند.  \nمسئله بازار :\n۱- کاهش فروش \n به\u200cدلیل نبود پاسخگویی به\u200cموقع و از دست دادن ۱۰٪\u200c تا ۴۰٪ مشتریان در ساعات مرده فروش و ایام  تعطیل\n2- کاهش بهره\u200cوری و افزایش خطا در پاسخگویی\nپشتیبانی مداوم و طولانی\u200cمدت سبب ایجاد خستگی  و افزایش خطا در عملکرد نیروی انسانی می شود. \n۳- منعطف نبودن ربات های پاسخگوی فعلی\n \nاستفاده از دستیار هوشمند کسب وکار رایابات که قادر به پاسخگویی مشتری، مشابه با لحن انسانی ، با قابلیت ۲۴/۷ در ساعات مرده فروش و ن

In [81]:
clean_text = re.sub(r'\s+', ' ', text.replace('\u200c', ' ')).strip()
clean_text

'معرفی سرویس ابری رایابات: دستیار هوشمند فروش گفتگومحور مبتنی بر مدل های زبانی بزرگ شرکت شما به دنبال تحول در فرآیندهای فروش و تعامل با مشتریان است؟ رایابات به عنوان یک سرویس ابری پیشرفته، امکان طراحی و پیاده سازی عامل های هوشمند فروش گفتگومحور را با استفاده از آخرین فناوری های پردازش زبان طبیعی (NLP) و مدل های زبانی بزرگ (LLM) فراهم می کند. این راهکار، نه تنها تعامل با مشتریان را بهینه می سازد، بلکه با ابزارهای تحلیلی قدرتمند و داشبوردهای کاربردی، مدیریت فرآیند فروش را متحول می کند. مسئله بازار : ۱- کاهش فروش به دلیل نبود پاسخگویی به موقع و از دست دادن ۱۰٪ تا ۴۰٪ مشتریان در ساعات مرده فروش و ایام تعطیل 2- کاهش بهره وری و افزایش خطا در پاسخگویی پشتیبانی مداوم و طولانی مدت سبب ایجاد خستگی و افزایش خطا در عملکرد نیروی انسانی می شود. ۳- منعطف نبودن ربات های پاسخگوی فعلی استفاده از دستیار هوشمند کسب وکار رایابات که قادر به پاسخگویی مشتری، مشابه با لحن انسانی ، با قابلیت ۲۴/۷ در ساعات مرده فروش و نیز ارائه تحلیل رفتار مشتری از خرید جهت بازنگری در استراتژی های فروش. ویژگی های رایابات: هوش مص

In [82]:
len(clean_text)

3637

In [97]:
test = clean_text[0:49]
print(test)

if (test[-1] !=' '):
    last_space = test.rfind(' ')
    test = test[:last_space]

print(test)

معرفی سرویس ابری رایابات: دستیار هوشمند فروش گفتگ
معرفی سرویس ابری رایابات: دستیار هوشمند فروش


In [98]:
len(clean_text)

3637

In [100]:
len(clean_text)%500

137

In [115]:
#Hyperparameters
chunk_len = 500    #512 =>1526
overlap   = 0.2

In [ ]:
def chunk_text(text, max_len, overlap):
    chunks = []
    start = 0
    text_len = len(text)
    overlap_len = int(max_len * overlap)

    while start < text_len:
        end = start + max_len
        #if last parageraph longer than text len 
        if end >= text_len:
            chunk = text[start:]
            chunks.append(chunk.strip())
            break
        
        #find a last space for start and end a chunk
        last_space = text.rfind(' ', start, end)
        if last_space == -1 or last_space <= start:
            last_space = end

        chunk = text[start:last_space].strip()
        chunks.append(chunk)

        # find a last space for overlab and starting a new chunk
        start = last_space - overlap_len
        if start < 0:
            start = 0
        else:
            
            prev_space = text.rfind(' ', 0, start)
            if prev_space != -1:
                start = prev_space + 1  

    return chunks

In [116]:
chunks = chunk_text(clean_text, max_len=chunk_len, overlap=overlap)
df = pd.DataFrame({'chunk': chunks})
df['length'] = df['chunk'].apply(len)
df

,chunk,length
0,معرفی سرویس ابری رایابات: دستیار هوشمند فروش گ...,499
1,بلکه با ابزارهای تحلیلی قدرتمند و داشبوردهای ک...,498
2,هوشمند کسب وکار رایابات که قادر به پاسخگویی مش...,498
3,لحن گفتار صمیمانه مدیریت چت مشتریان امکان استف...,491
4,مشتریان تعریف کنند. ورود کاربران: کاربران با ل...,496
5,توسط صاحبان کسب وکار، خدمات بهینه ای را برای ک...,497
6,(KPI) مانند نرخ تبدیل، زمان پاسخگویی و رضایت م...,496
7,گسترده ای از مشتریان. ۴. افزایش نرخ تبدیل و فر...,499
8,مشتریان در قیف فروش: از استعلام اولیه تا تکمیل...,475


In [117]:
print('chunks len :',sum(df['length']))
print('data len :', len(clean_text))

chunks len : 4449
data len : 3637


In [ ]:
model =SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2', cache_folder='./DistiluseBaseMultilingualCasedV2')

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Batches:   0%|          | 0/1 [00:00<?, ?it/s]c:\Users\LavaN-32428068\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\distilbert\modeling_distilbert.py:402: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Batches: 100%|██████████| 1/1 [00:03<00:00,  3.62s/it]


In [119]:
embeddings = model.encode(chunks, show_progress_bar=True)

Batches: 100%|██████████| 1/1 [00:01<00:00,  1.29s/it]


In [121]:
df['embeddings'] = list(embeddings)
df

,chunk,length,embeddings
0,معرفی سرویس ابری رایابات: دستیار هوشمند فروش گ...,499,"[0.02429498, -0.030774668, -0.012917013, -0.00..."
1,بلکه با ابزارهای تحلیلی قدرتمند و داشبوردهای ک...,498,"[0.06964124, -0.0034599546, -0.02729405, 0.036..."
2,هوشمند کسب وکار رایابات که قادر به پاسخگویی مش...,498,"[0.030626599, -0.013502225, -0.007562358, 0.03..."
3,لحن گفتار صمیمانه مدیریت چت مشتریان امکان استف...,491,"[0.019158846, -0.028246813, 0.032721914, 0.005..."
4,مشتریان تعریف کنند. ورود کاربران: کاربران با ل...,496,"[0.014599482, -0.018233193, 0.0080176415, -0.0..."
5,توسط صاحبان کسب وکار، خدمات بهینه ای را برای ک...,497,"[0.068489835, -0.0060667135, -0.003688835, 0.0..."
6,(KPI) مانند نرخ تبدیل، زمان پاسخگویی و رضایت م...,496,"[0.030834582, 0.023667129, 0.019583747, 0.0213..."
7,گسترده ای از مشتریان. ۴. افزایش نرخ تبدیل و فر...,499,"[0.045921527, 0.0085801575, 0.016428929, 0.025..."
8,مشتریان در قیف فروش: از استعلام اولیه تا تکمیل...,475,"[0.028499965, -0.047942974, -0.013819949, -0.0..."


In [122]:
dim = embeddings[0].shape[0]
print(dim)
index = faiss.IndexFlatL2(dim)
index.add(np.array(embeddings))
print('for_search :',index)

512
for_search : <faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x000001B2B3DA1170> >


In [132]:
query_embedding = model.encode([user_query])[0]
query_embedding

array([ 3.03132292e-02,  9.74856783e-03, -5.19566722e-02,  4.52660397e-02,
        8.37186202e-02,  3.03910952e-02,  1.08558862e-02, -1.24455327e-02,
        9.19696409e-03,  4.59422767e-02,  1.33708864e-02,  1.51553135e-02,
       -2.22908016e-02, -1.57126074e-03, -1.76815037e-02, -2.39873994e-02,
        7.58087263e-02,  2.97352802e-02,  4.13509421e-02,  1.72231731e-03,
       -3.41599509e-02, -1.11139324e-02,  2.53811013e-02, -3.01177171e-03,
       -3.49879675e-02, -6.40181266e-03, -4.95395111e-03, -2.65398398e-02,
       -1.41335446e-02, -1.32789593e-02, -2.09620930e-02,  3.59795094e-02,
       -2.68691331e-02, -1.17334370e-02, -4.78283763e-02,  4.73290496e-02,
       -1.62640419e-02,  7.18526263e-03, -7.73384189e-03, -4.91053388e-02,
       -3.43602598e-02, -8.47130641e-03,  3.29496190e-02,  7.59445783e-03,
       -3.68747348e-03,  1.92479929e-03,  4.43317927e-02, -3.02448086e-02,
        6.23887070e-02,  3.71902734e-02, -2.25641113e-02, -1.38238622e-02,
        1.39422584e-02,  

In [133]:
D, I = index.search(np.array([query_embedding]), k=3)
retrieved_chunks = [chunks[i] for i in I[0]]
retrieved_chunks

['توسط صاحبان کسب وکار، خدمات بهینه ای را برای کاربران به ارمغان می آورد. توانایی تشخیص صوت، پردازش زبان و پاسخگویی دقیق، رایابات را به دستیاری هوشمند و قابل اعتماد برای کسب وکارها تبدیل کرده است. مزیت های رقابتی رایابات: ۱. داشبوردهای هوشمند و کاربردی: - نظارت لحظه ای بر عملکرد تیم فروش و تعاملات مشتریان. - تحلیل رفتار مشتریان و شناسایی فرصت های فروش با گزارش های خودکار. - نمایش معیارهای کلیدی (KPI) مانند نرخ تبدیل، زمان پاسخگویی و رضایت مشتری. ۲. اتوماسیون فرآیندهای فروش: - پاسخگویی خودکار به',
 'مشتریان در قیف فروش: از استعلام اولیه تا تکمیل فرآیند خرید. - تبلیغات هوشمند: ارائه پیشنهادهای شخصی سازی شده در زمان مناسب. رایابات تنها یک ربات گفتگو نیست، یک همکار هوشمند فروش است که با ترکیب فناوری هوش مصنوعی و تحلیل داده های عملیاتی ، به کسب وکار شما کمک می کند تا در رقابت بازار پیشتاز باشد. اطلاعات تماس شرکت : آدرس وبسایت : www.rayabot.ir و شماره تماس : 09157797315 و شبکه اجتماعی : @rp_datacience شعارها : هر کسب و کار یک دستیار هوشمند هوش مصنوعی در خدمت فروش هوشمند!',
 'معرفی سرویس ابری

In [ ]:
def cosine_similarity(vec1, vec2):
    """محاسبه شباهت کسینوسی بین دو بردار"""
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))


def retrieve_best_chunk(df, user_embedding):

    best_score = -1
    best_chunk = None

    for i, row in df.iterrows():
        chunk_embedding = np.array(row['embeddings'])  
        score = cosine_similarity(user_embedding, chunk_embedding)

        if score > best_score:
            best_score = score
            best_chunk = row['chunk']

    return best_chunk, best_score

In [130]:
user_query = "مزیت رقابتی رایابات "

In [134]:
result, score = retrieve_best_chunk(df, query_embedding)
print("بهترین پاسخ:", result)
print("شباهت:", score)

بهترین پاسخ: توسط صاحبان کسب وکار، خدمات بهینه ای را برای کاربران به ارمغان می آورد. توانایی تشخیص صوت، پردازش زبان و پاسخگویی دقیق، رایابات را به دستیاری هوشمند و قابل اعتماد برای کسب وکارها تبدیل کرده است. مزیت های رقابتی رایابات: ۱. داشبوردهای هوشمند و کاربردی: - نظارت لحظه ای بر عملکرد تیم فروش و تعاملات مشتریان. - تحلیل رفتار مشتریان و شناسایی فرصت های فروش با گزارش های خودکار. - نمایش معیارهای کلیدی (KPI) مانند نرخ تبدیل، زمان پاسخگویی و رضایت مشتری. ۲. اتوماسیون فرآیندهای فروش: - پاسخگویی خودکار به
شباهت: 0.25829586
